In [1]:
import yaml
import MetaTrader5 as mt5
import pandas as pd
import os
import numpy as np

In [2]:
def max_drawdown(equity_curve, verbose=True):
    # Calcular el running max de la equity curve
    running_max = np.maximum.accumulate(equity_curve)
    
    # Calcular el drawdown
    drawdown = (equity_curve - running_max) / running_max
    
    # Encontrar el valor máximo de drawdown y la fecha correspondiente
    max_drawdown_value = np.min(drawdown) * 100  # Convertir el drawdown a porcentaje
    max_drawdown_date = equity_curve.index[np.argmin(drawdown)]
    
    if verbose:
        print(f"Máximo drawdown: {max_drawdown_value:.2f}%")
        print(f"Fecha del máximo drawdown: {max_drawdown_date}")

    return max_drawdown_value

In [3]:
INITIAL_CASH = 10_000

leverages_per_category = {
    'Equities I CFD': 1,
    'Forex': 30,
    'Cash II CFD': 9,
}


In [4]:
import MetaTrader5 as mt5

# Inicializar MetaTrader 5
if not mt5.initialize():
    print("Error al inicializar MT5")
    quit()

pd.set_option('display.max_columns', 500) # number of columns to be displayed


if not mt5.initialize():
    print("initialize() failed, error code =",mt5.last_error())
    quit()

root = './backbone/data'

with open('configs/live_trading.yml', 'r') as file:
    strategies = yaml.safe_load(file)

timeframes_to_number = {
    'H1': 16385,
    'H2': 16386,
    'H3': 16387,
    'H4': 16388,
}

all_equity_curves = {}
all_trades = {}

for strategy_name, configs in strategies.items():
    
    instruments_info = configs['instruments_info']
    wfo_params = configs['wfo_params']
    opt_params = configs['opt_params']
    name = strategy_name.split('.')[-1]
    use_wfo = wfo_params['use_wfo']
    
    
    root =  os.path.join('./backtesting_pipeline/strategies', name)
    
    if use_wfo:
        dir = os.path.join(root, 'full_analysis')
    
    else:
        dir = os.path.join(root, 'preliminar_analysis')
        
    for ticker, info in instruments_info.items():
        symbol_info = mt5.symbol_info(ticker)
        category = symbol_info.path.split('\\')[0]
        
        leverage = leverages_per_category[category]
        
        timeframe = timeframes_to_number[info['timeframe']]
        key = f'{ticker}_{timeframe}'
        
        full_key = f'{name}_{ticker}_{timeframe}'
        
        # if full_key in candidates:
        equity = pd.read_csv(
            os.path.join(dir, key, 'equity.csv'), index_col=0
        )
        equity.index = pd.to_datetime(equity.index)
        
        trades = pd.read_csv(
            os.path.join(dir, key, 'trades.csv'), usecols=['Size', 'EntryPrice', 'ExitPrice', 'PnL', 'EntryTime', 'ExitTime']
        )
        trades.EntryTime = pd.to_datetime(trades.EntryTime)
        trades.ExitTime = pd.to_datetime(trades.ExitTime)
        
        trades['margin'] = (np.abs(trades['Size']) * trades['EntryPrice']) / leverage
        
        all_equity_curves[full_key] = equity
        all_trades[full_key] = trades

In [5]:
risks = [0.5, 1.5]
all_syntetic_trades = {}
all_syntetic_eq_curves = {}

for strategy in all_trades.keys():
    ticker = strategy.split('_')[1] # <-- Creo que esto podria hacerse mejor
    
    symbol_info = mt5.symbol_info(ticker)
    category = symbol_info.path.split('\\')[0]
    
    leverage = leverages_per_category[category]
    
    for risk in risks:
        trades = all_trades[strategy]
        eq_curve = all_equity_curves[strategy]

        # trades = all_trades['Channel_NVDA_16387']
        # eq_curve = all_equity_curves['Channel_NVDA_16387']

        trades = pd.merge(
            trades,
            eq_curve['Equity'],
            left_on='ExitTime',
            right_index=True,
            how='inner'
        )

        trades['ReturnPct'] = trades['PnL'] / trades['Equity'].shift(1)
        trades.loc[0, 'ReturnPct'] = trades.loc[0, 'PnL'] / INITIAL_CASH
        trades['margin'] = (np.abs(trades['Size']) * trades['EntryPrice']) / leverage

        syntetic_trades = trades.copy()

        syntetic_trades[['ReturnPct', 'Size']] = syntetic_trades[['ReturnPct', 'Size']] * risk

        syntetic_trades['Equity'] = INITIAL_CASH * (1 + syntetic_trades['ReturnPct']).cumprod()

        syntetic_trades['PnL'] = syntetic_trades['Equity'] - syntetic_trades['Equity'].shift(1)
        syntetic_trades.loc[0, 'PnL'] = syntetic_trades.loc[0, 'Equity'] - INITIAL_CASH
        syntetic_trades['margin'] = (np.abs(syntetic_trades['Size']) * syntetic_trades['EntryPrice']) / leverage
    
        all_syntetic_trades[f'{strategy}_r{risk}'] = syntetic_trades
        all_syntetic_eq_curves[f'{strategy}_r{risk}'] = syntetic_trades.Equity


# Cálculo del margen

In [6]:
import pandas as pd


# Convertir columnas a datetime
for df in all_trades.values():
    df["EntryTime"] = pd.to_datetime(df["EntryTime"])
    df["ExitTime"] = pd.to_datetime(df["ExitTime"])

# Concatenar y calcular los eventos
all_events = pd.concat([
    pd.concat([
        df[["EntryTime", "margin"]].rename(columns={"EntryTime": "time", "margin": "change"}).round(3),
        df[["ExitTime", "margin"]].rename(columns={"ExitTime": "time", "margin": "change"}).assign(change=lambda x: -x["change"]).round(3)
    ]) for df in all_trades.values()
])

# Ordenar por tiempo
all_events = all_events.sort_values(['time', 'change'], ascending=[True, False]).reset_index(drop=True)

# Calcular el margen acumulado
all_events["margin"] = all_events["change"].cumsum()

all_events

# agregar la columna del equity general (portfolio) y restarle el margen disponible

,time,change,margin
0,2021-01-25 18:00:00+00:00,546.859,5.468590e+02
1,2021-01-26 14:00:00+00:00,-546.859,0.000000e+00
2,2021-01-28 13:00:00+00:00,455.606,4.556060e+02
3,2021-01-28 17:00:00+00:00,-455.606,0.000000e+00
4,2021-02-08 06:00:00+00:00,1243.933,1.243933e+03
...,...,...,...
1609,2024-10-31 20:00:00+00:00,490.457,1.685154e+03
1610,2024-11-01 00:00:00+00:00,1523.716,3.208870e+03
1611,2024-11-01 00:00:00+00:00,-490.457,2.718413e+03
1612,2024-11-01 00:00:00+00:00,-1194.697,1.523716e+03


In [7]:
# Curva de equity general calculada con distinto riesgo en cada bot
# Margin acumulado en cada momento del tiempo de esa curva de equity general
# valor del percentil 90 del margin y cantidad de veces que lo supero
# Ver si en mt5 puedo traer el tipo de activo que es cada ticker, para ver cuanto leverage ponerle
# Ver si tengo que reescalar algo de los precios para calcular el margin
# agregar un parametro de risk por cada combinacion de activo-bot para permitir que corran con diferentes riesgos
# Columna con cantidad de margin calls y step outs

In [8]:
import pandas as pd

min_date = None
max_date = None

for name, curve in all_equity_curves.items():
    # Convertir las fechas a UTC si son tz-naive
    actual_date = curve.index[0].tz_localize('UTC') if curve.index[0].tz is None else curve.index[0].tz_convert('UTC')
    
    # Si min_date es None, inicializar con la primera fecha
    if min_date is None:
        min_date = actual_date
    # Comparar si la fecha actual es menor que min_date
    elif actual_date < min_date:
        min_date = actual_date

    # Si max_date es None, inicializar con la última fecha
    curve_last_date = curve.index[-1].tz_localize('UTC') if curve.index[-1].tz is None else curve.index[-1].tz_convert('UTC')
    
    if max_date is None:
        max_date = curve_last_date
    # Comparar si la fecha actual es mayor que max_date
    elif curve_last_date > max_date:
        max_date = curve_last_date

# Mostrar las fechas encontradas
print(f"Min Date: {min_date}")
print(f"Max Date: {max_date}")

# Calcular min_date y max_date
min_date = min_date.date()
max_date = max_date.date()

print(min_date)
print(max_date)

date_range = pd.to_datetime(pd.date_range(start=min_date, end=max_date, freq='D'))
print(date_range)

Min Date: 2021-01-04 00:00:00+00:00
Max Date: 2024-11-01 00:00:00+00:00
2021-01-04
2024-11-01
DatetimeIndex(['2021-01-04', '2021-01-05', '2021-01-06', '2021-01-07',
               '2021-01-08', '2021-01-09', '2021-01-10', '2021-01-11',
               '2021-01-12', '2021-01-13',
               ...
               '2024-10-23', '2024-10-24', '2024-10-25', '2024-10-26',
               '2024-10-27', '2024-10-28', '2024-10-29', '2024-10-30',
               '2024-10-31', '2024-11-01'],
              dtype='datetime64[ns]', length=1398, freq='D')


In [9]:

def get_hipotetical_wallet_equity(equity_curves, initial_equity):
    total = pd.DataFrame()

    for name, curve in equity_curves.items():
        eq = curve.copy()
        eq = eq.reset_index().rename(columns={'index':'Date'})[['Date','Equity']].sort_values(by='Date')
        eq['Date'] = pd.to_datetime(eq['Date'])
        eq['Date'] = eq['Date'].dt.floor('D').dt.date

        eq = eq.groupby('Date').agg({'Equity':'last'})

        eq = eq.reindex(date_range)
        
        eq.Equity = eq.Equity.ffill()
        eq.Equity = eq.Equity.fillna(INITIAL_CASH)
    
        eq['variacion'] = eq['Equity'] - eq['Equity'].shift(1)
        eq['variacion_porcentual'] = eq['variacion'] / eq['Equity'].shift(1)
        
        df_variacion = pd.DataFrame(
            {
                f'variacion_{name}': eq.variacion_porcentual.fillna(0)
            }
        )
        
        total = pd.concat([total, df_variacion], axis=1)

    total = total.reset_index().rename(columns={'index':'Date'})

    # Inicializa el valor de equity
    total['Equity'] = initial_equity

    # Lista de columnas con las variaciones porcentuales
    variation_cols = [col for col in total.columns if col.startswith('variacion')]

    # Calcular la curva de equity
    for i in range(1, len(total)):
        previous_equity = total.loc[i-1, 'Equity']  # Equity del periodo anterior
        
        # Calcula el impacto monetario de cada bot por separado y suma el resultado
        impact_sum = 0
        for col in variation_cols:
            variation = total.loc[i, col]
            impact_sum += previous_equity * variation
        
        # Actualiza el equity sumando el impacto monetario total
        total.loc[i, 'Equity'] = previous_equity + impact_sum

    # Resultado final

    total = total.set_index('Date')
    return total[['Equity']]

if 'variaciones_porcentuales' in all_equity_curves.keys():
    del all_equity_curves['variaciones_porcentuales']

total = get_hipotetical_wallet_equity(equity_curves=all_equity_curves, initial_equity=INITIAL_CASH)
all_equity_curves['variaciones_porcentuales'] = total

In [10]:
variaciones_porcentuales = all_equity_curves['variaciones_porcentuales']
variaciones_porcentuales.index = pd.to_datetime(variaciones_porcentuales.index)

In [11]:
all_events['time'] = pd.to_datetime(all_events['time']).dt.date
all_events.set_index('time', inplace=True)

In [12]:
all_events = pd.merge(
    all_events,
    variaciones_porcentuales,
    left_index=True,
    right_index=True,
    how='left'
)

all_events

,change,margin,Equity
2021-01-25,546.859,5.468590e+02,10003.379752
2021-01-26,-546.859,0.000000e+00,10005.059752
2021-01-28,455.606,4.556060e+02,10045.376377
2021-01-28,-455.606,0.000000e+00,10045.376377
2021-02-08,1243.933,1.243933e+03,10092.240698
...,...,...,...
2024-10-31,490.457,1.685154e+03,23054.013827
2024-11-01,1523.716,3.208870e+03,23037.686787
2024-11-01,-490.457,2.718413e+03,23037.686787
2024-11-01,-1194.697,1.523716e+03,23037.686787


In [13]:
all_events = all_events.round(2)

all_events['margin_level'] = ((all_events['Equity'] / all_events['margin']) * 100)

all_events['free_margin'] = (all_events['Equity'] - all_events['margin'])

all_events

,change,margin,Equity,margin_level,free_margin
2021-01-25,546.86,546.86,10003.38,1829.239659,9456.52
2021-01-26,-546.86,0.00,10005.06,inf,10005.06
2021-01-28,455.61,455.61,10045.38,2204.819912,9589.77
2021-01-28,-455.61,0.00,10045.38,inf,10045.38
2021-02-08,1243.93,1243.93,10092.24,811.318965,8848.31
...,...,...,...,...,...
2024-10-31,490.46,1685.15,23054.01,1368.068718,21368.86
2024-11-01,1523.72,3208.87,23037.69,717.937779,19828.82
2024-11-01,-490.46,2718.41,23037.69,847.469293,20319.28
2024-11-01,-1194.70,1523.72,23037.69,1511.937233,21513.97


In [14]:
all_events.to_csv("./margins.csv",index=False)

In [21]:
margin_calls = all_events[(all_events['margin_level'] < 100)&(all_events['margin_level'] != -1*np.inf)]

margin_calls

,change,margin,Equity,margin_level,free_margin


In [22]:
import plotly.graph_objects as go

# Crear una figura vacía
fig = go.Figure()

# Recorrer las curvas de equity de cada bot y agregarlas al gráfico
fig.add_trace(go.Scatter(x=all_events.index, y=all_events.Equity, mode='lines', name='Equity'))
fig.add_trace(go.Scatter(x=margin_calls.index, y=margin_calls.Equity, mode='markers', name='Margin calls'))
fig.add_trace(go.Scatter(x=all_events.index, y=all_events.margin, mode='lines', name='margin'))
fig.add_trace(go.Scatter(x=all_events.index, y=all_events.margin_level, mode='lines', name='margin_level'))
fig.add_trace(go.Scatter(x=all_events.index, y=all_events.free_margin, mode='lines', name='free_margin'))

# Agregar una línea horizontal en y=100
fig.add_shape(
    type="line",
    x0=all_events.index.min(),
    x1=all_events.index.max(),
    y0=100,
    y1=100,
    line=dict(color="red", width=2, dash="dash"),  # Color rojo, línea discontinua
    name="y=100"
)

# Actualizar los detalles del layout del gráfico
fig.update_layout(
    title="Curva de Margin de Múltiples Bots",
    xaxis_title="Fecha",
    yaxis_title="Equity",
    legend_title="Bots",
    shapes=[  # Asegurar que la línea se incluya en el layout
        dict(
            type="line",
            x0=all_events.index.min(),
            x1=all_events.index.max(),
            y0=100,
            y1=100,
            line=dict(color="red", width=2, dash="dash")
        )
    ]
)

# Mostrar el gráfico
fig.show()

In [16]:
import plotly.graph_objects as go

# Crear una figura vacía
fig = go.Figure()

# Recorrer las curvas de equity de cada bot y agregarlas al gráfico
    
fig.add_trace(go.Scatter(x=all_events.index, y=all_events.Equity, mode='lines', name='Equity'))
fig.add_trace(go.Scatter(x=margin_calls.index, y=margin_calls.Equity, mode='markers', name='Margin calls'))
fig.add_trace(go.Scatter(x=all_events.index, y=all_events.margin, mode='lines', name='margin'))
fig.add_trace(go.Scatter(x=all_events.index, y=all_events.margin_level, mode='lines', name='margin_level'))
fig.add_trace(go.Scatter(x=all_events.index, y=all_events.free_margin, mode='lines', name='free_margin'))


# Actualizar los detalles del layout del gráfico
fig.update_layout(
    title="Curva de Margin de Múltiples Bots",
    xaxis_title="Fecha",
    yaxis_title="Equity",
    legend_title="Bots"
)

# Mostrar el gráfico
fig.show()

In [17]:
import plotly.graph_objects as go

# Crear una figura vacía
fig = go.Figure()

# Recorrer las curvas de equity de cada bot y agregarlas al gráfico
    
fig.add_trace(go.Scatter(x=all_events.index, y=all_events.margin_level, mode='lines', name='margin_level'))

# Actualizar los detalles del layout del gráfico
fig.update_layout(
    title="Curva de Margin de Múltiples Bots",
    xaxis_title="Fecha",
    #yaxis_title="Equity",
    legend_title="Bots"
)

# Mostrar el gráfico
fig.show()

In [18]:
0/0

ZeroDivisionError: division by zero

In [ ]:
import plotly.graph_objects as go

# Crear una figura vacía
fig = go.Figure()

# Recorrer las curvas de equity de cada bot y agregarlas al gráfico
for k, v in all_equity_curves.items():
    
    fig.add_trace(go.Scatter(x=v.index, y=v.Equity, mode='lines', name=k))

# Actualizar los detalles del layout del gráfico
fig.update_layout(
    title="Curvas de Equity de Múltiples Bots",
    xaxis_title="Fecha",
    yaxis_title="Equity",
    legend_title="Bots"
)

# Mostrar el gráfico
fig.show()

max_drawdown(all_equity_curves['variaciones_porcentuales'])


In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px

# Supongamos que tienes el DataFrame `df` como en el ejemplo
# Convertir el índice a tipo datetime si no lo está
df = all_equity_curves["variaciones_porcentuales"]
df.index = pd.to_datetime(df.index)

# Calcular los retornos diarios en porcentaje
df['Daily Return'] = ((df['Equity'] - df['Equity'].shift(1)) / df['Equity'].shift(1)) * 100

# Crear un DataFrame resampleado con valores mínimo y máximo para cada mes
monthly_min = df['Daily Return'].resample("M").min()
monthly_max = df['Daily Return'].resample("M").max()

# Aplicar la lógica de np.where para seleccionar el mínimo si < 0, máximo si >= 0
monthly_returns = np.where(monthly_max >= 0, monthly_max, monthly_min)

# Crear un índice temporal basado en las fechas de los datos mensuales
monthly_index = df['Daily Return'].resample("M").apply(lambda x: x.index[-1])

# Crear un DataFrame para el gráfico
monthly_df = pd.DataFrame({
    'Fecha': monthly_index,
    'Retorno Mensual': monthly_returns
})

display(monthly_df)

# Crear el gráfico de barras con Plotly
fig = px.bar(
    monthly_df,
    x='Fecha',
    y='Retorno Mensual',
    labels={"Fecha": "Fecha", "Retorno Mensual": "Retorno Mensual (%)"},
    title="Retornos Mensuales Ajustados en Porcentaje",
    text_auto='.2s'
)

fig.update_layout(
    # width=800,
    # height=700,
    xaxis_title="Fecha",
    yaxis_title="Retorno Mensual (%)",
    xaxis=dict(tickformat="%Y-%m"),
)

fig.show()


In [11]:
del all_equity_curves['variaciones_porcentuales']

# FTMO Simulator

In [12]:
def ftmo_simulator(equity_curve, initial_cash):
    equity_curve['month'] = pd.to_datetime(equity_curve.index)
    equity_curve['month'] = pd.to_datetime(equity_curve['month'], errors='coerce')  # Asegúrate de que sea datetime
    equity_curve['month'] = equity_curve['month'].dt.to_period('M')  # Convertir a un periodo mensual
    
    equity_curve.fillna(0, inplace=True)

    # Identificar índices de los valores máximo y mínimo por mes
    max_indices = equity_curve.groupby('month')['Equity'].idxmax()
    min_indices = equity_curve.groupby('month')['Equity'].idxmin()

    # Combinar índices únicos
    unique_indices = pd.Index(max_indices).union(pd.Index(min_indices))

    equity_curve = equity_curve.loc[unique_indices]
    
    # Inicializar acumuladores globales
    total_positive_hits = 0
    total_negative_hits = 0
    all_time_to_positive = []
    all_time_to_negative = []

    # Simulación para cada mes como punto de partida
    for i in range(0, len(equity_curve), 2):
        perc_change = 0
        time_to_positive = []
        time_to_negative = []
        
        actual_equity = equity_curve.iloc[i].Equity
        
        months_elapsed = 0

        # Iterar desde el mes de inicio hacia adelante
        for j in range(i, len(equity_curve)):
            
            future_equity = equity_curve.iloc[j].Equity
            
            if i == 0 and j == 0:
                perc_change = ((future_equity - initial_cash) / initial_cash) * 100
                
            else:
                perc_change = ((future_equity - actual_equity) / actual_equity) * 100
            
            months_elapsed += 0.5

            if perc_change >= 10:
                total_positive_hits += 1
                time_to_positive.append(months_elapsed)
                months_elapsed = 0
                break

            elif perc_change <= -10:
                total_negative_hits += 1
                time_to_negative.append(months_elapsed)
                months_elapsed = 0
                break

        # Guardar tiempos de esta simulación
        all_time_to_positive.extend(time_to_positive)
        all_time_to_negative.extend(time_to_negative)

    return total_positive_hits, total_negative_hits, all_time_to_positive, all_time_to_negative


In [13]:
# ftmo_simulator(equity_curves['variaciones_porcentuales'], 10_000)

In [ ]:
from itertools import combinations

# Generar todas las combinaciones de las claves del diccionario
keys = list(all_equity_curves.keys())

equity_curves_combinations = []

# Obtenemos combinaciones desde tamaño 1 hasta len(keys)
for r in range(2, len(keys) + 1):
    for combination in combinations(keys, r):
        equity_curves_combinations.append(combination)

equity_curves_combinations

In [ ]:

from sklearn.linear_model import LinearRegression

stability_ratio = 0
best_ratio = 0

results = pd.DataFrame()

for combination in equity_curves_combinations:
    selected_curves = {}
    for name in combination:
        selected_curves[name] = all_equity_curves[name]
    
    # Generar la curva combinada
    combined_equity = get_hipotetical_wallet_equity(selected_curves, initial_equity=INITIAL_CASH)
    
    # Calcular métricas como retorno y drawdown
    x = np.arange(len(combined_equity)).reshape(-1, 1)
    reg = LinearRegression().fit(x, combined_equity)
    stability_ratio = reg.score(x, combined_equity)
    
    return_ = ((combined_equity.Equity.iloc[-1] - combined_equity.Equity.iloc[0]) / combined_equity.Equity.iloc[0]) * 100
    dd = -1 * max_drawdown(combined_equity, verbose=False)
    
    positive_hits, negative_hits, time_to_positive, time_to_negative = ftmo_simulator(combined_equity, INITIAL_CASH)

    combination_results = pd.DataFrame({
        'combination':[combination],
        'stability_ratio':[stability_ratio],
        'return':[return_],
        'drawdown':[dd],
        'ret_drawdown':[return_/dd],
        
        'success':[positive_hits / (positive_hits + negative_hits)],
        'positive_hits':[positive_hits],
        'negative_hits':[negative_hits],
        'mean_time_to_positive':[np.mean(time_to_positive)],
        'std_time_to_positive':[np.std(time_to_positive)],
        'mean_time_to_negative':[np.mean(time_to_negative)],
        'std_time_to_negative':[np.std(time_to_negative)],
        
    })

    results = pd.concat([results, combination_results])
    
results.sort_values(by='ret_drawdown', ascending=False)

In [15]:
# results.to_csv('./wallet_results.csv')

In [ ]:

0/0

In [16]:
# ShortIBS_ADAUSD_H4 = equity_curves['ShortIBS_ADAUSD_H4']
# ShortIBS_DOTUSD_H4 = equity_curves['ShortIBS_DOTUSD_H4']
# TripleST_NVDA_H4 = equity_curves['TripleST_NVDA_H4']
# MeanRev_XMRUSD_H4 = equity_curves['MeanRev_XMRUSD_H1']

# del equity_curves['TripleST_NVDA_H4']
# del equity_curves['TripleSMA_US100.cash_H2']
# del equity_curves['TripleST_TSLA_H3']


# del equity_curves['MeanRev_XMRUSD_H1']


# equity_curves['ShortIBS_ADAUSD_H4'] = ShortIBS_ADAUSD_H4
# equity_curves['ShortIBS_DOTUSD_H4'] = ShortIBS_DOTUSD_H4
# equity_curves['TripleST_NVDA_H4'] = TripleST_NVDA_H4
# equity_curves['MeanRev_XMRUSD_H4'] = MeanRev_XMRUSD_H4

# Correlaciones

In [ ]:
percentual_differences = {}

def get_percentual_differences(equity_curves):
    for name, curve in equity_curves.items():
        equity_df = curve.copy()
        
        equity_df = equity_df.reset_index().rename(columns={'index':'Date'})
        equity_df['Date'] = pd.to_datetime(equity_df['Date'])
        equity_df['Date'] = equity_df['Date'].dt.floor('D').dt.date
        
        equity_df = pd.DataFrame(equity_df.groupby('Date')['Equity'].last()).reindex(date_range)
        equity_df.ffill(inplace=True)
        equity_df.fillna(INITIAL_CASH, inplace=True)

        equity_df['diff'] = equity_df['Equity'] - equity_df['Equity'].shift(1)
        
        percentual_differences[name] = equity_df
    
    return percentual_differences


differences = get_percentual_differences(all_equity_curves)
differences

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Paso 1: Unir todas las curvas de equity en un solo DataFrame basado en la fecha
all_equity_df = pd.DataFrame()

for name, df in differences.items():
    all_equity_df[name] = df.resample('M').agg({'Equity':'last','diff':'sum',})['diff']


# Paso 2: Calcular la matriz de correlación

# all_equity_df = all_equity_df[['BPercent_IQm_H1','BPercent_NTESm_H1']]
correlation_matrix = all_equity_df.corr(method='pearson')

# Paso 3: Plotear el mapa de calor de correlación usando seaborn
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation Matrix of Equity Curves')
plt.xticks(rotation=75)
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Paso 1: Unir todas las curvas de equity en un solo DataFrame basado en la fecha
all_equity_df = pd.DataFrame()

for name, df in differences.items():
    all_equity_df[name] = df.resample('W').agg({'Equity':'last','diff':'sum',})['diff']


# Paso 2: Calcular la matriz de correlación

# all_equity_df = all_equity_df[['BPercent_IQm_H1','BPercent_NTESm_H1']]
correlation_matrix = all_equity_df.corr(method='pearson')

# Paso 3: Plotear el mapa de calor de correlación usando seaborn
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation Matrix of Equity Curves')
plt.xticks(rotation=75)
plt.show()